In [1]:
%load_ext autoreload
%autoreload 2
from diacritization import recodex_predict
from diacritization import Dataset
import numpy as np
import lzma
import pickle
from diacritization import flatten
import numpy as np

In [2]:
data = Dataset()
with lzma.open("diacritization.model", "rb") as model_file:
    nn = pickle.load(model_file)

with lzma.open("onehot.encoder", "rb") as model_file:
    ohe = pickle.load(model_file)

with lzma.open("label.encoder", "rb") as model_file:
    le = pickle.load(model_file)

In [23]:
def find_ngrams(s, n):
    return zip(*[s[i:] for i in range(n)])

input_len = 13
features = np.array([
        np.array(
            list(
                find_ngrams(
                    "".join(["#" for i in range(int((input_len - 1) / 2))])
                    + sentence.lower()
                    + "".join(["#" for i in range(int((input_len - 1) / 2))]),
                    input_len,
                )
            )
        )
        for sentence in data.data.translate(Dataset.DIA_TO_NODIA).split("\n")[:-1]
    ])

In [4]:
features = ohe.transform(features)

In [16]:
predictions = nn.predict(features[:200000])

In [18]:
predictions = le.inverse_transform(predictions)

In [22]:
len(data.data.strip("\n").strip(" "))

563360

In [23]:
len(features)

553990

In [ ]:

def capitalize_word(accented_word, capitalized_word):
    try:
        final_capitalized_word = []
        for accented_character, capitalized_character in zip(
            accented_word, capitalized_word
        ):
            if capitalized_character.isupper():
                final_capitalized_word.append(accented_character.upper())
            else:
                final_capitalized_word.append(accented_character)
        return "".join(final_capitalized_word)
    except:
        return capitalized_word

def sentence_predict(sentence_sliding_window, sentence_orig, nn, ohe, le):
    try:
        ohe_sentence = ohe.transform(sentence_sliding_window)
        predictions = nn.predict(ohe_sentence)
        characters = le.inverse_transform(predictions)
        if len(characters) != len(sentence_orig):
            return sentence_orig
        sentence_predicted = "".join(
            np.where(
                (np.array(list(characters)) == " ")
                & (np.array(list(sentence_orig)) != " "),
                np.array(list(sentence_orig)),
                np.array(list(characters)),
            )
        )
        words_predicted = sentence_predicted.split(" ")
        words_orig = sentence_orig.split(" ")
        words_capitalized = [
            capitalize_word(accented_word, orig_word)
            for accented_word, orig_word in zip(words_predicted, words_orig)
        ]
        return " ".join(words_capitalized)
    except:
        return sentence_orig

predicted_sentences = [
    sentence_predict(transformed_sentence, orig_sentence, nn, ohe, le)
    for transformed_sentence, orig_sentence in zip(features, sentences)
]

predictions = "\n".join(predicted_sentences)
return predictions

In [79]:
with open("gold.txt", "w") as text_file:
    text_file.write(data.data)

In [80]:
with open("pred.txt", "w") as text_file:
    text_file.write(predictions)